# Programación asincrónica en Python

La *programación asincrónica* es una forma de organizar las llamadas a los métodos de una aplicación, de forma tal que aquellos métodos que están esperando por una operación que no requiere procesamiento (escribir en un archivo, conectarse a una base de datos, recibir datos de una página web) no bloquean a los demás métodos que pueden ser ejecutados.

Python empieza a introducir las primeras nociones de programación asincrónica en su versión 3.4 (Marzo 2014) con la introducción del módulo [asyncio](https://docs.python.org/3.4/library/asyncio.html). En su siguiente versión (3.5, Septiembre 2015), se simplificó la sintaxis para definir corutinas (la base de la programación asincrónica en Python). Al momento esta notación continúa siendo revisada y podría sufrir algunos cambios a futuro.

** Requisitos para ejecutar este notebook **
* Python 3.5

## Evolución de la programación asicrónica en Python

El siguiente video y su correspondiente [documento](http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html), muestran cómo fue evolucionando la sintaxis de programación asincrónica en Python, mediante un simple ejemplo explicado nada menos que por Guido Van Rossum y Jesse Jiryu Davis (MongoDB). Es una buena forma de entender qué problemas intenta resolver la programación asincrónica, recomiendo su lectura y visur... visualización :) (es sábado a la mañana, sepan disculpar)

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("7sCu4gEjH5I")

### Conceptos básicos (extraídos del video)

Definiciones básicas de la programación asincrónica que simplificarán su entendimiento:

* *non-blocking method*: método que retorna "inmediatamente" (al contexto que lo invocó) sin haber finalizado la ejecución de su código y luego avisa (mediante un evento) cuando esta ha finalizado.
* *callback*: función que se asocia a un evento, para ser ejecutada al momento que este se produce.
* *event loop*: loop que detecta los eventos y ejecuta los callback asociados.

Con los tres conceptos anteriores (más alguna forma de definir y detectar los eventos que queremos que ejecuten los callbacks) ya podíamos hacer programación asincrónica en Python.

Como se podrá ver en le video trabajar con esos conceptos produce una sintaxis bastante desordenada, por eso se introduce el concepto de *corutina* (coroutine). Una corutina es una subrutina que puede pausarse a sí misma (por ejemplo al momento de requerir una operación de IO) y posteriormente ser reanudada (desde ese mismo punto). Esto permite a las corutinas coordinarse entre sí para no bloquearse al momento de realizar operaciones de IO.

En Python las corutinas se implementan usando *generators*, por lo que es conveniente saber cómo estos funcionan, para entender cómo las corutinas delegan el control a la función que las invocó.

Más conceptos que se añaden con corutinas:
* *Future*: representa a un resultado pendiente que será obtenido cuando se produzca un evento por el cuál estamos esperando. Almacena los callbacks que serán ejecutados cuando este evento suceda.
* *Task*: objeto que organiza la ejecución de una coroutine. 

Hint: Task es subclase de Future.

## Ventajas sobre threads
Estas son algunas ventajas de las corutinas sobre los threads

* Consumen menos memoria
* No están limitadas por por configuraciones del usuario o el SO
* A diferencia de los threads, permiten indicar en que partes de su ejecución prodrán ser interrumpidas.
* Generan un stacktrace más simple y claro de entender.

## Ejemplo Simple

A continuación voy a mostrar un ejemplo simple de la notación para definir corutinas en Python.

In [1]:
import asyncio

En Python 3.5 se agrega la sintaxis de *async* y *await* para definir corutinas (ver [PEP-492](https://www.python.org/dev/peps/pep-0492/))

In [15]:
async def my_coroutine(task_name, seconds_to_sleep=3):
    print('{0} sleeping for: {1} seconds'.format(task_name, seconds_to_sleep))
    await asyncio.sleep(seconds_to_sleep)
    print('{0} is finished'.format(task_name))

En este caso, estamos definiendo que la corutina *my_coroutine* (con el statement *async*), que pausará su ejecución cuando se alcance el comando *await*. Las corutinas no son métodos comunes, por ejemplo al invocarlas no se ejecuta su código sino que se obtiene un objeto corutina.

In [21]:
c = my_coroutine("corrutina_ejemplo")
c

/home/santiago/miniconda3/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: coroutine 'my_coroutine' was never awaited
  if __name__ == '__main__':


<coroutine object my_coroutine at 0x7f799039dca8>

Podemos crear varias de estas corrutinas y ejecutarlas en paralelo usando un *event loop*.

In [22]:
def run_tasks():
    loop = asyncio.get_event_loop()
    tasks = [
        my_coroutine('task1', 1),
        my_coroutine('task2', 1),
        my_coroutine('task3', 1)]
    loop.run_until_complete(asyncio.wait(tasks))
    # loop.close()

In [17]:
%timeit -n1 -r1 run_tasks()

task2 sleeping for: 1 seconds
task3 sleeping for: 1 seconds
task1 sleeping for: 1 seconds
task2 is finished
task3 is finished
task1 is finished
1 loops, best of 1: 1 s per loop


Como se ve, el tiempo total de ejecución de las tareas es 1 segundo (las tareas se ejecutan en paralelo).

Si estamos trabajando con Python 3.4 la corutina se define con la anotación *@asyncio.coroutine* (en lugar de *async*) y *yield from* (en lugar de *await*)

In [23]:
@asyncio.coroutine
def my_coroutine_3_4(task_name, seconds_to_sleep=3):
    print('{0} sleeping for: {1} seconds'.format(task_name, seconds_to_sleep))
    yield from asyncio.sleep(seconds_to_sleep)
    print('{0} is finished'.format(task_name))